<a href="https://colab.research.google.com/github/kmnsky/DS_Homework/blob/master/Groupby.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# data analysis and wrangling
import pandas as pd
import numpy as np
import re

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Mounting GoogleDrive
from google.colab import drive
drive.mount('/content/drive')

# Writing multiple dataframes to worksheets using Pandas and XlsxWriter
!pip install xlsxwriter
import xlsxwriter

Mounted at /content/drive
     |████████████████████████████████| 153kB 7.4MB/s 


In [ ]:
# Отображение 2-х знаков после запятой
#pd.options.display.float_format = '{:,.2f}'.format

In [ ]:
# fbtool = pd.read_excel("/content/drive/My Drive/Datasets/Статистика аккаунтов_01_06_12.xlsx", dtype = str)
# crm = pd.read_excel("/content/drive/My Drive/Datasets/crm_01_06_12.xlsx", dtype = str)

In [ ]:
xls = pd.ExcelFile("/content/drive/My Drive/Datasets/Статистика аккаунтов_05_10.xlsx")
fbtool = pd.read_excel(xls, 'Sheet1')
crm = pd.read_excel(xls, 'Sheet2')

### **Fbtool**

In [ ]:
# Изменение header
fbtool.columns = fbtool.iloc[0]
fbtool = fbtool.reindex(fbtool.index.drop(0))
cols = list(fbtool.columns)
cols[1] = 'Utm_campaign'
cols[2] = 'Сabinet'
cols[3] = 'Cost'
cols[4] = 'Show'
cols[5] = 'Link_cliks'
cols[6] = 'Install'
fbtool.columns = cols

In [ ]:
# Удаление NaN
fbtool = fbtool.replace(0,np.nan).dropna(axis=1,how="all")

In [ ]:
# Удаление колонок
fbtool.drop(["Сabinet", "CPI", "CPC", "CPM", "CTR"], axis=1, inplace=True )

In [ ]:
# удалить данные после пробела
fbtool['Utm_campaign'] = fbtool['Utm_campaign'].map(lambda x: x.split()[0])

In [ ]:
print(fbtool['Utm_campaign'].values)

['OPD_DE_SL_LPkPU200per_SOC3545_adsnew58525_V_camp2_adset1_2_auto_mw_1865_tx130_Despy702d38a_MythicalSphinx_And2_A2'
 'OPD_DE_SL_LPkPU200per_SOC3545_adsnew58521_V_camp2_adset1_2_auto_mw_1865_tx130_Despy702d38a_MythicalSphinx_And2_A2'
 'OPD_DE_SL_LPkPU200per_SOC3616_adsnew59800_V_camp1_adset1_2_auto_mw_1865_notx_Despy702d38a_DevilsCat_And2_A2'
 'OPD_DE_SL_LPkPU200per_SOC2063_adsnew59963_V_camp1_adset1_2_auto_mw_1865_tx130_Despy702d38a_MythicalSphinx_And2_A2'
 'OPD_DE_SL_LPkPU200per_SOC2063_adsnew60059_V_camp1_adset1_2_auto_mw_1865_tx130_Despy702d38a_MythicalSphinx_And2_A2'
 'OPD_DE_SL_LPkPU200per_SOC2884_adsnew60760_V_camp1_adset1_2_auto_mw_1865_tx130_Despy702d38a_MythicalSphinx_And2_A2'
 'OPD_DE_SL_LPkPU200per_SOC2884_adsnew60765_V_camp1_adset1_2_auto_mw_1865_tx130_Despy702d38a_MythicalSphinx_And2_A2'
 'OPD_DE_SL_LPkPU200per_SOC2884_adsnew60676_V_camp2_adset1_2_auto_mw_1865_tx130_Despy702d38a_MythicalSphinx_And2_A2'
 'OPD_DE_SL_LPkPU200per_SOC3651_adsnew61094_V_camp1_adset1_2_auto_mw_1

In [ ]:
# текст по столбцам
#fb_addset = fbtool['Utm_campaign'].str.split('_', expand=True)
fb_addset = fbtool['Utm_campaign'].str.split(r'\_|\-', expand=True)

In [ ]:
# Header for adset
fb_addset.columns=['adset', 'geo', 'brand', 'page', 'soc', 
                   'bm', 'acc', 'camp', 'adset1', 'int', 'placement', 
                   'gender', 'age', 'text', 'creo', 'app', 'os', 'mb']

In [ ]:
fb_addset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23 entries, 1 to 23
Data columns (total 18 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   adset      23 non-null     object
 1   geo        23 non-null     object
 2   brand      23 non-null     object
 3   page       23 non-null     object
 4   soc        23 non-null     object
 5   bm         23 non-null     object
 6   acc        23 non-null     object
 7   camp       23 non-null     object
 8   adset1     23 non-null     object
 9   int        23 non-null     object
 10  placement  23 non-null     object
 11  gender     23 non-null     object
 12  age        23 non-null     object
 13  text       23 non-null     object
 14  creo       23 non-null     object
 15  app        23 non-null     object
 16  os         23 non-null     object
 17  mb         23 non-null     object
dtypes: object(18)
memory usage: 3.4+ KB


In [ ]:
# соединить два датафрейма по столбцам
fbtool_final = pd.concat([fbtool,fb_addset],axis=1)

In [ ]:
# Изменение форматов
fbtool_final['Cost'] = fbtool_final['Cost'].astype(float)
fbtool_final['Show'] = fbtool_final['Show'].astype(int)
fbtool_final['Link_cliks'] = fbtool_final['Link_cliks'].astype(int)
fbtool_final['Install'] = fbtool_final['Install'].astype(int)

### **Настройка времени и ACC метки**

In [ ]:
# Время
fbtool_final['Time'] = '10:00'

In [ ]:
# ACC только V
fbtool_final = fbtool_final.loc[fbtool_final.acc == 'V']

## **CRM**

In [ ]:
# Удаление колонок
crm.drop(["Utm_source", 'Utm_content', "Utm_medium"], axis=1, inplace=True)

In [ ]:
# Переименование колонок
crm = crm.rename(columns=
                       {'Registration count': 'Reg',
                        'Ftd count': 'Ftd'
                       })

In [ ]:
# замена прочерков
crm['Ftd'].replace('-', 0, inplace=True)

In [ ]:
# Изменение форматов
crm['Reg'] = crm['Reg'].astype(int)
crm['Ftd'] = crm['Ftd'].astype(int)

## **Combining**

In [ ]:
# Left join
df = fbtool_final.merge(crm, how='left', on=["Utm_campaign"])

In [ ]:
# Замена NaN
df.fillna(0, inplace=True)

In [ ]:
# Изменение форматов
df['Reg'] = df['Reg'].astype(int)

## **GROUP BY**

In [ ]:
# Группировка при выборке
df_group = pd.DataFrame(df.groupby(['bm', 'Utm_campaign'], as_index=False).sum())

In [ ]:
# Вычисление
df_group = df_group.assign(Instal_cost = df_group.Cost/df_group.Install)
df_group = df_group.assign(Reg_cost = df_group.Cost/df_group.Reg)
df_group = df_group.assign(CTR = df_group.Link_cliks/df_group.Show*100)
df_group = df_group.assign(FTD_cost = df_group.Cost/df_group.Ftd)

In [ ]:
# Удаление колонок
df_group.drop(['Show', 'Link_cliks'], axis=1, inplace=True)

In [ ]:
# Замена Inf
df_group.replace([np.inf, -np.inf], 0, inplace=True)

In [ ]:
# Словарь класификации
codes_cats = df_group.Utm_campaign.astype('category')
dict_cats = dict(enumerate(codes_cats.cat.categories))

In [ ]:
# Присвоенные cat.codes
df_group['ID'] = df_group.Utm_campaign.astype('category').cat.codes

# Розшифровка
#df_group['level_back'] = df_group['ID'].map(dict_cats)

### **Modeling**

In [ ]:
# machine learning
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score , accuracy_score, precision_score
from sklearn.model_selection import train_test_split
from xgboost.sklearn import XGBClassifier

In [ ]:
df_train = pd.read_excel("/content/drive/My Drive/Datasets/Model_data_multi_test.xlsx")

In [ ]:
df_test = df_group[['ID', 'Cost', 'Install', 'Reg', 'Instal_cost', 'Reg_cost']].copy()

In [ ]:
# Переименование и удаление колонок
df_train.drop(['Ftd', 'FTD_cost'], axis=1, inplace=True)

df_train = df_train.rename(columns=
                       {'Name': 'ID',})

In [ ]:
X = df_train.drop("Target",axis=1)
y = df_train["Target"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=42)

In [ ]:
#Using kfold cross validation
from sklearn.model_selection import KFold, cross_val_predict
kf = KFold(shuffle=True, random_state=42, n_splits=3)
for train_index, test_index in kf.split(X):
    X_train, X_test, y_train, y_test = (X.iloc[train_index, :], 
                                        X.iloc[test_index, :], 
                                        y[train_index], 
                                        y[test_index])

In [ ]:
from xgboost.sklearn import XGBClassifier
xgb=XGBClassifier(learning_rate=0.001, objective='multi:softprob', n_estimators=2500,
                                max_depth=4, min_child_weight=0,
                                gamma=0, subsample=0.7,
                                colsample_bytree=0.7,
                                scale_pos_weight=1, seed=27,
                                reg_alpha=0.00006)
xgb.fit(X_train,y_train)
predict_12=xgb.predict(X_test)

In [ ]:
print(accuracy_score(y_test,predict_12))
print(confusion_matrix(y_test,predict_12))
print(classification_report(y_test,predict_12))

0.9724137931034482
[[158   4   1]
 [  1  71   0]
 [  0   2  53]]
              precision    recall  f1-score   support

           0       0.99      0.97      0.98       163
           1       0.92      0.99      0.95        72
           2       0.98      0.96      0.97        55

    accuracy                           0.97       290
   macro avg       0.97      0.97      0.97       290
weighted avg       0.97      0.97      0.97       290



In [ ]:
pred = xgb.predict(df_test)
pred = pred.astype(np.int)

## **Analysis**

**Total page**

In [ ]:
submission = pd.DataFrame({'ID':df_test['ID'],'score':pred})

In [ ]:
total = pd.merge(df_group, 
                   submission,
                   on = 'ID')

In [ ]:
total = total[['bm', 'Utm_campaign', 'Cost', 'Install', 'Reg', 
               'Ftd', 'Instal_cost', 'Reg_cost', 'FTD_cost', 'CTR', 'score']]

In [ ]:
# Два знака после запятой
total = total.round(decimals=2)

In [ ]:
# Сортировка Cost по возростанию
total = total.sort_values(['Cost'], ascending=False)

In [ ]:
# Замена 0 на -
total['Cost'].replace(0, '-', inplace=True)
total['Install'].replace(0, '-', inplace=True)
total['Reg'].replace(0, '-', inplace=True)
total['Ftd'].replace(0, '-', inplace=True)
total['Instal_cost'].replace(0, '-', inplace=True)
total['Reg_cost'].replace(0, '-', inplace=True)
total['FTD_cost'].replace(0, '-', inplace=True)
total['Cost'] = total['Cost'].astype(int)

In [ ]:
# # Загрузить файл
# total.to_excel('AdsetPredicted.xlsx', index=False)
# from google.colab import files
# files.download('AdsetPredicted.xlsx')

### **Time page**

In [ ]:
df_time = pd.DataFrame(df[['Time', 'Cost','Link_cliks', 'Show', 'Install', 'Reg', 'Ftd']])

In [ ]:
df_time = df_time.groupby('Time', as_index=False).sum().copy()

In [ ]:
# Вычисление
df_time = df_time.assign(Instal_cost = df_time.Cost/df_time.Install)
df_time = df_time.assign(Reg_cost = df_time.Cost/df_time.Reg)
df_time = df_time.assign(FTD_cost = df_time.Cost/df_time.Ftd)
df_time = df_time.assign(CTR = df_time.Link_cliks/df_time.Show*100)

In [ ]:
# Замена Inf
df_time.replace([np.inf, -np.inf], 0, inplace=True)

In [ ]:
# Два знака после запятой
df_time = df_time.round(decimals=2)

In [ ]:
# Удалить колонки 
df_time = df_time.drop(['Link_cliks', 'Show'], axis=1)

In [ ]:
# Замена 0 на -
df_time['Install'].replace(0, '-', inplace=True)
df_time['Reg'].replace(0, '-', inplace=True)
df_time['Instal_cost'].replace(0, '-', inplace=True)
df_time['Reg_cost'].replace(0, '-', inplace=True)
df_time['Ftd'].replace(0, '-', inplace=True)
df_time['FTD_cost'].replace(0, '-', inplace=True)

### **Creo page**

In [ ]:
df_creo = pd.DataFrame(df[['creo', 'Cost','Link_cliks', 'Show', 'Install', 'Reg', 'Ftd']])

In [ ]:
df_creo = df_creo.groupby('creo', as_index=False).sum().sort_values(['Cost'], ascending=False)

In [ ]:
# Вычисление
df_creo = df_creo.assign(Instal_cost = df_creo.Cost/df_creo.Install)
df_creo = df_creo.assign(Reg_cost = df_creo.Cost/df_creo.Reg)
df_creo = df_creo.assign(FTD_cost = df_creo.Cost/df_creo.Ftd)
df_creo = df_creo.assign(CTR = df_creo.Link_cliks/df_creo.Show*100)

In [ ]:
# Замена Inf
df_creo.replace([np.inf, -np.inf], 0, inplace=True)

In [ ]:
# # Добавление столбца итогов в фрейм данных Pandas
# df_creo = df_creo.append(pd.Series(df_creo.sum(),name='Total'))
# df_creo['creo'].replace(df_creo['creo']['Total'], 'Total', inplace=True)

In [ ]:
# Два знака после запятой
df_creo = df_creo.round(decimals=2)

In [ ]:
# Удалить колонки 
df_creo = df_creo.drop(['Link_cliks', 'Show'], axis=1)

In [ ]:
# Замена 0 на -
df_creo['Install'].replace(0, '-', inplace=True)
df_creo['Reg'].replace(0, '-', inplace=True)
df_creo['Instal_cost'].replace(0, '-', inplace=True)
df_creo['Reg_cost'].replace(0, '-', inplace=True)
df_creo['Ftd'].replace(0, '-', inplace=True)
df_creo['FTD_cost'].replace(0, '-', inplace=True)

In [ ]:
df_creo

,creo,Cost,Install,Reg,Ftd,Instal_cost,Reg_cost,FTD_cost,CTR
0,DEvid700d26b,195.49,71,14,2,2.75,13.96,97.75,0.94
1,Despy702d38a,176.34,25,12,-,7.05,14.70,-,1.27


### **App page**

In [ ]:
df_app = pd.DataFrame(df[['app', 'Cost','Link_cliks', 'Show', 'Install', 'Reg', 'Ftd']])

In [ ]:
df_app = df_app.groupby('app', as_index=False).sum().sort_values(['Cost'], ascending=False)

In [ ]:
# Вычисление
df_app = df_app.assign(Instal_cost = df_app.Cost/df_app.Install)
df_app = df_app.assign(Reg_cost = df_app.Cost/df_app.Reg)
df_app = df_app.assign(FTD_cost = df_app.Cost/df_app.Ftd)
df_app = df_app.assign(CTR = df_app.Link_cliks/df_app.Show*100)

In [ ]:
# Замена Inf
df_app.replace([np.inf, -np.inf], 0, inplace=True)

In [ ]:
# # Добавление столбца итогов в фрейм данных Pandas
# df_app = df_app.append(pd.Series(df_app.sum(),name='Total'))
# df_app['app'].replace(df_app['app']['Total'], 'Total', inplace=True)

In [ ]:
# Два знака после запятой
df_app = df_app.round(decimals=2)

In [ ]:
# Удалить колонки 
df_app = df_app.drop(['Link_cliks', 'Show'], axis=1)

In [ ]:
# Замена 0 на -
df_app['Install'].replace(0, '-', inplace=True)
df_app['Reg'].replace(0, '-', inplace=True)
df_app['Instal_cost'].replace(0, '-', inplace=True)
df_app['Reg_cost'].replace(0, '-', inplace=True)
df_app['Ftd'].replace(0, '-', inplace=True)
df_app['FTD_cost'].replace(0, '-', inplace=True)

In [ ]:
# Активные компании
#active_adset = total.query('score == 1').copy()
#active_adset = active_adset['bm']
#print('Active')
#for i in set(active_adset):
#  print(i)

In [ ]:
# Перезапуск
#restart_adset = total.query('score == 0').copy()
#restart_adset = restart_adset['bm']
#print('Перезапуск')
#for i in set(restart_adset):
#  print(i)

### **Information page**

In [ ]:
df_info = df[['Time', 'Utm_campaign', 'Cost', 'Show', 'Link_cliks', 
          'Install', 'Reg', 'Ftd', 'adset', 'geo', 'brand', 'page', 
          'soc', 'bm', 'acc', 'camp', 'adset1', 'int', 'placement', 
          'gender', 'age', 'text', 'creo', 'app', 'os', 'mb']].copy()

### **Save file**

In [ ]:
writer = pd.ExcelWriter('DE_10-00.xlsx', engine='xlsxwriter')

# Write each dataframe to a different worksheet.
total.to_excel(writer, sheet_name='Total_DE', index=False)
df_time.to_excel(writer, sheet_name='Time_DE', index=False)
df_creo.to_excel(writer, sheet_name='Creo_DE', index=False)
df_app.to_excel(writer, sheet_name='App_DE', index=False)
df_info.to_excel(writer, sheet_name='Information', index=False)

# Get the xlsxwriter workbook and worksheet objects.
workbook  = writer.book
worksheet_total = writer.sheets['Total_DE']
worksheet_time = writer.sheets['Time_DE']
worksheet_creo = writer.sheets['Creo_DE']
worksheet_app = writer.sheets['App_DE']

# Set the column width and format.
worksheet_total.set_column('A:A', 18)
worksheet_total.set_column('B:B', 70)

worksheet_time.set_column('A:A', 18)
worksheet_creo.set_column('A:A', 18)
worksheet_app.set_column('A:A', 18)


# Close the Pandas Excel writer and output the Excel file.
writer.save()

In [ ]:
# Загрузить файл
from google.colab import files
files.download('DE_10-00.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# df1 = pd.DataFrame([
# {'Time': '10:00', 'Id': 1, 'Cost': 25},
# {'Time': '10:00', 'Id': 2, 'Cost': 15}
# ])

# df2 = pd.DataFrame([
# {'Time': '11:00', 'Id': 1, 'Cost': 30},
# {'Time': '11:00', 'Id': 2, 'Cost': 18}
# ])

# df = df1.merge(df2,  on=['Id'], suffixes=('_x', ''))

# df['Cost'] = df['Cost'] - df['Cost_x']

# df_final = df1.append(df.loc[:, ['Time', 'Id', 'Cost']])